* Starting with bigquery and accessing our first database 

In [1]:
from google.cloud import bigquery

In [2]:
client = bigquery.Client()

c:\Users\ramse\DSPROJ\.venv\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [3]:
dataset_ref = client.dataset("chicago_crime", project="bigquery-public-data")

In [4]:
crime_data = client.get_dataset(dataset_ref)

In [5]:
#list all the tables in the chicago crime data 

tables = list(client.list_tables(crime_data))

#print names of all tables in the dastaset 

for table in tables: 
    print(table.table_id)

crime


In [6]:
table_ref = dataset_ref.table("crime")

table = client.get_table(table_ref)

In [7]:
#print information on all columns in the "crime" table in teh "chicago_crime" dataset 
table.schema

[SchemaField('unique_key', 'INTEGER', 'REQUIRED', None, None, (), None),
 SchemaField('case_number', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('date', 'TIMESTAMP', 'NULLABLE', None, None, (), None),
 SchemaField('block', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('iucr', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('primary_type', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('description', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('location_description', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('arrest', 'BOOLEAN', 'NULLABLE', None, None, (), None),
 SchemaField('domestic', 'BOOLEAN', 'NULLABLE', None, None, (), None),
 SchemaField('beat', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('district', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('ward', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('community_area', 'INTEGER', 'NULLABLE', None, None, (), None),
 

In [8]:
client.list_rows(table, max_results=5).to_dataframe()

,unique_key,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,location
0,12126240,JD321197,2020-08-05 01:00:00+00:00,0000X E LAKE ST,0261,CRIMINAL SEXUAL ASSAULT,AGGRAVATED - HANDGUN,SIDEWALK,False,False,...,42,32,02,1176513.0,1901749.0,2020,2020-08-12 15:41:51+00:00,41.885752,-87.627261,"(41.885752167, -87.627260665)"
1,5737132,HN538883,2007-08-19 17:30:00+00:00,002XX S CANAL ST,0312,ROBBERY,ARMED:KNIFE/CUTTING INSTRUMENT,OTHER RAILROAD PROP / TRAIN DEPOT,False,False,...,2,28,03,1173177.0,1899236.0,2007,2018-02-28 15:56:25+00:00,41.878931,-87.639586,"(41.878931004, -87.639585621)"
2,10042597,HY230989,2015-04-21 20:19:00+00:00,001XX N MICHIGAN AVE,0312,ROBBERY,ARMED:KNIFE/CUTTING INSTRUMENT,CONVENIENCE STORE,False,False,...,42,32,03,1177271.0,1901522.0,2015,2018-02-10 15:50:01+00:00,41.885112,-87.624484,"(41.885112119, -87.624484053)"
3,13604042,JH436313,2024-09-20 01:30:00+00:00,001XX N STATE ST,0312,ROBBERY,ARMED - KNIFE / CUTTING INSTRUMENT,CTA TRAIN,False,False,...,42,32,03,1176330.0,1901649.0,2024,2024-12-21 15:40:46+00:00,41.885482,-87.627936,"(41.885481891, -87.627935689)"
4,9624766,HX274673,2014-05-24 05:00:00+00:00,001XX N DEARBORN ST,0312,ROBBERY,ARMED:KNIFE/CUTTING INSTRUMENT,STREET,False,False,...,42,32,03,1175930.0,1900878.0,2014,2016-02-04 06:33:39+00:00,41.883375,-87.629428,"(41.883375237, -87.629427772)"


In [9]:
query = """
    SELECT MIN(year) as earliest_year
    FROM `bigquery-public-data.chicago_crime.crime`
"""
query_job = client.query(query)
results = query_job.result()

for row in results:
    print(f"The earliest year in the dataset is: {row.earliest_year}")

The earliest year in the dataset is: 2001


In [10]:
timestamp_columns = [field.name for field in table.schema if field.field_type == 'TIMESTAMP']
print(f"Number of TIMESTAMP columns: {len(timestamp_columns)}")
print("TIMESTAMP columns:", timestamp_columns)

Number of TIMESTAMP columns: 2
TIMESTAMP columns: ['date', 'updated_on']


In [11]:
num_timestamp = [field.name for field in table.schema if field.field_type == 'TIMESTAMP']

In [12]:
#SQL query to find crimes only related to robbery
query = """
    SELECT primary_type, description, date
    FROM `bigquery-public-data.chicago_crime.crime`
    WHERE primary_type = 'ROBBERY'
    """
#create a QueryJobConfig to estimate the size of a query without running it 

dry_run_config = bigquery.QueryJobConfig(dry_run = True) 

#API request - dry run query to estimate costs 
dry_run_query_job = client.query(query, job_config = dry_run_config)

print("this query will be processed using {} bytes." .format(dry_run_query_job.total_bytes_processed))

#query_job = client.query(query)

#robbery = query_job.to_dataframe()

#we can specify a param when running a query to limit how much data we're willing to scan

#lets say itll run if its less than 1GB

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 1000 * 1000)

safe_query_job = client.query(query,  job_config = safe_config)

safe_query_job.to_dataframe()

this query will be processed using 315524795 bytes.


c:\Users\ramse\DSPROJ\.venv\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,primary_type,description,date
0,ROBBERY,ARMED - OTHER DANGEROUS WEAPON,2024-06-26 18:00:00+00:00
1,ROBBERY,ARMED - OTHER DANGEROUS WEAPON,2023-12-31 02:00:00+00:00
2,ROBBERY,ATTEMPT ARMED - OTHER DANGEROUS WEAPON,2022-06-24 12:05:00+00:00
3,ROBBERY,ARMED: OTHER FIREARM,2017-04-13 21:30:00+00:00
4,ROBBERY,ATTEMPT STRONG ARM - NO WEAPON,2022-03-22 11:30:00+00:00
...,...,...,...
309880,ROBBERY,ARMED - KNIFE / CUTTING INSTRUMENT,2021-04-29 18:00:00+00:00
309881,ROBBERY,ARMED - KNIFE / CUTTING INSTRUMENT,2021-12-09 18:55:00+00:00
309882,ROBBERY,ARMED - KNIFE / CUTTING INSTRUMENT,2022-06-04 20:52:00+00:00
309883,ROBBERY,ARMED - KNIFE / CUTTING INSTRUMENT,2024-07-01 15:00:00+00:00


In [37]:

import pandas as pd

robbery

NameError: name 'robbery' is not defined

* SELECT 

use of the SELECT function to filter different results from a large dataset

In [38]:


#referene to the openaq dataset
dataset_ref = client.dataset("openaq", project = "bigquery-public-data")

#API request to fesch the dataset
pollution_data = client.get_dataset(dataset_ref)

#lets list all the tables in the pollution database 

tables = list(client.list_tables(pollution_data))

for table in tables: 
    print(table.table_id)

#looks like theres only one table in this database called global_air_quality 

#need a reference to the global_air_quality table ]
aq_table = client.get_table(dataset_ref.table("global_air_quality"))


global_air_quality


In [39]:
#lets check out the details of the dataset 

aq_table.schema 

[SchemaField('location', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('city', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('country', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('pollutant', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('value', 'FLOAT', 'NULLABLE', None, None, (), None),
 SchemaField('timestamp', 'TIMESTAMP', 'NULLABLE', None, None, (), None),
 SchemaField('unit', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('source_name', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('latitude', 'FLOAT', 'NULLABLE', None, None, (), None),
 SchemaField('longitude', 'FLOAT', 'NULLABLE', None, None, (), None),
 SchemaField('averaged_over_in_hours', 'FLOAT', 'NULLABLE', None, None, (), None),
 SchemaField('location_geom', 'GEOGRAPHY', 'NULLABLE', None, None, (), None)]

In [41]:
#lets check out the first few rows of the dataset 


client.list_rows(aq_table, max_results = 5).to_dataframe()

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours,location_geom
0,"Borówiec, ul. Drapałka",Borówiec,PL,bc,0.85217,2022-04-28 07:00:00+00:00,µg/m³,GIOS,1.0,52.276794,17.074114,POINT(52.276794 1)
1,"Kraków, ul. Bulwarowa",Kraków,PL,bc,0.91284,2022-04-27 23:00:00+00:00,µg/m³,GIOS,1.0,50.069308,20.053492,POINT(50.069308 1)
2,"Płock, ul. Reja",Płock,PL,bc,1.41000,2022-03-30 04:00:00+00:00,µg/m³,GIOS,1.0,52.550938,19.709791,POINT(52.550938 1)
3,"Elbląg, ul. Bażyńskiego",Elbląg,PL,bc,0.33607,2022-05-03 13:00:00+00:00,µg/m³,GIOS,1.0,54.167847,19.410942,POINT(54.167847 1)
4,"Piastów, ul. Pułaskiego",Piastów,PL,bc,0.51000,2022-05-11 05:00:00+00:00,µg/m³,GIOS,1.0,52.191728,20.837489,POINT(52.191728 1)


In [42]:
#lets write a query to select all the data from Ropczyce in PL
poland_query = """
    SELECT *
    from `bigquery-public-data.openaq.global_air_quality`
    WHERE city = 'Kraków'
    """

dry_run_config = bigquery.QueryJobConfig(dry_run = True)

dry_run_query_job = client.query(poland_query, job_config = dry_run_config)

print("query will take {} bytes".format(dry_run_query_job.total_bytes_processed))


poland_query = client.query(poland_query).to_dataframe()

poland_query

query will take 773830128 bytes


c:\Users\ramse\DSPROJ\.venv\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours,location_geom
0,"Kraków, ul. Bujaka",Kraków,PL,bc,0.16164,2022-05-11 11:00:00+00:00,µg/m³,GIOS,1.0,50.010575,19.949189,POINT(50.010575 1)
1,"Kraków, ul. Bujaka",Kraków,PL,bc,1.43280,2022-04-29 06:00:00+00:00,µg/m³,GIOS,1.0,50.010575,19.949189,POINT(50.010575 1)
2,"Kraków, ul. Bujaka",Kraków,PL,bc,0.78678,2022-05-18 03:00:00+00:00,µg/m³,GIOS,1.0,50.010575,19.949189,POINT(50.010575 1)
3,"Kraków, ul. Bujaka",Kraków,PL,bc,0.31185,2022-05-17 03:00:00+00:00,µg/m³,GIOS,1.0,50.010575,19.949189,POINT(50.010575 1)
4,"Kraków, ul. Bujaka",Kraków,PL,bc,0.44082,2022-05-10 06:00:00+00:00,µg/m³,GIOS,1.0,50.010575,19.949189,POINT(50.010575 1)
...,...,...,...,...,...,...,...,...,...,...,...,...
9522,"Kraków, ul. Bulwarowa",Kraków,PL,pm25,17.98280,2022-05-06 07:00:00+00:00,µg/m³,GIOS,1.0,50.069308,20.053492,POINT(50.069308 1)
9523,"Kraków, ul. Bulwarowa",Kraków,PL,pm25,10.15460,2022-05-07 11:00:00+00:00,µg/m³,GIOS,1.0,50.069308,20.053492,POINT(50.069308 1)
9524,"Kraków, ul. Bulwarowa",Kraków,PL,pm25,11.57490,2022-05-07 10:00:00+00:00,µg/m³,GIOS,1.0,50.069308,20.053492,POINT(50.069308 1)
9525,"Kraków, ul. Bulwarowa",Kraków,PL,pm25,10.72470,2022-04-30 15:00:00+00:00,µg/m³,GIOS,1.0,50.069308,20.053492,POINT(50.069308 1)


In [ ]:
#we want countries that work in ppm 
query = """
        SELECT DISTINCT country 
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE UNIT = 'ppm'
        """
#the dry run config is already defined, so we can runa  dry run config query to see how 
#big this is gonna be 
dry_run = poland_client.query(query, job_config = dry_run_config)

print("this query will take {} bytes".format(dry_run.total_bytes_processed))

#config that will only run the query if the limit is less than 10 GB
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10 ** 10)

#we can run our query, we can call .to_dataframe() on the query 
# to make it into a python dataframe so we can look at it!
ppm_data = client.query(query, job_config = safe_config).to_dataframe()

ppm_data


In [35]:
#run a query to find which data points had pollution of 0 

query = """
        SELECT * 
        from `bigquery-public-data.openaq.global_air_quality`
        WHERE value = 0
        """
#we want to see how big this job is going to be
size_config = bigquery.QueryJobConfig(dry_run = True) 

size_query = client.query(query, job_config = size_config)

print('this job will take {} bytes' .format(size_query.total_bytes_processed)) 

#now we can run safe query so we dont spend the big bucks parusing throught this
#random dataset
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10 ** 10)

zero_pollutants = client.query(query, job_config = safe_config).to_dataframe()



this job will take 773830128 bytes


c:\Users\ramse\DSPROJ\.venv\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [36]:
zero_pollutants

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours,location_geom
0,"Zielonka, Bory Tucholskie",Zielonka,PL,bc,0.0,2022-04-29 14:00:00+00:00,µg/m³,GIOS,1.0,53.662136,17.933986,POINT(53.662136 1)
1,"Toruń, ul. Przy Kaszowniku",Toruń,PL,bc,0.0,2022-04-19 04:00:00+00:00,µg/m³,GIOS,1.0,53.017628,18.612808,POINT(53.017628 1)
2,"Kielce, ul. Targowa",Kielce,PL,bc,0.0,2022-05-07 17:00:00+00:00,µg/m³,GIOS,1.0,50.878998,20.633692,POINT(50.878998 1)
3,"Zielonka, Bory Tucholskie",Zielonka,PL,bc,0.0,2022-05-19 14:00:00+00:00,µg/m³,GIOS,1.0,53.662136,17.933986,POINT(53.662136 1)
4,"Koszalin, ul. Armii Krajowej",Koszalin,PL,bc,0.0,2022-05-12 20:00:00+00:00,µg/m³,GIOS,1.0,54.193986,16.172544,POINT(54.193986 1)
...,...,...,...,...,...,...,...,...,...,...,...,...
192702,Leitrim,Fezile Dabi,ZA,pm25,0.0,2022-03-31 23:00:00+00:00,µg/m³,South Africa,1.0,-26.851461,27.875845,POINT(-26.851461 1)
192703,Bosjesspruit,Gert Sibande,ZA,pm25,0.0,2022-03-29 06:00:00+00:00,µg/m³,South Africa,1.0,-26.605556,29.210833,POINT(-26.605556 1)
192704,Stellenboch,Cape Winelands,ZA,pm25,0.0,2022-05-11 03:00:00+00:00,µg/m³,South Africa,1.0,-33.927762,18.857242,POINT(-33.927762 1)
192705,eMbalenhle North,Gert Sibande,ZA,pm25,0.0,2022-05-25 08:00:00+00:00,µg/m³,South Africa,1.0,-26.536389,29.072500,POINT(-26.536389 1)


In [43]:
dataset = client.get_dataset(client.dataset("hacker_news", project=
                                    "bigquery-public-data"))

## GROUP BY, HAVING , COUNT 

we can select raw data but now it looks like its time to group data and count things within grops

COUNT()

returns a count of things 

GROUP BY

takes the name of one or more columns and treates all rows with the same value in teh column as a single group when you apply aggregate functions 

HAVING


is used in combination with GROUP BY to ignore groups that dont meet certain criteria




## Example of COUNT()

In [ ]:
query = """  
        SELECT COUNT(ID)
        FROM bigquery-public-data .pet_records.pets
        """

#this would return 4 because there are 4 IDS in the 'pets' table

## Example of GROUP BY

In [ ]:
query = """   
        SELECT Animal, COUNT(ID)
        FROM 'bigquery-public-data.pet_records.pets'
        GROUP BY Animal
        """

#this would return a table with three rows (one for each different animal)
# so the pets table would contain 1 rabbit 1 dog and 2 cats

## Example of GROUP BY ... HAVING 

In [ ]:
query = """   
        SELECT Anmial, COUNT(ID)
        FROM 'bigquery-public-data.pet_records.pets' 
        GROUP BY Anmial 
        HAVING COUNT(ID) > 1 """

# this returns a table with 1 row, since only 1 group meets the
# criteria

In [ ]:
query = """    
        SELECT parent, COUNT(id)
        FROM 'bigquery-public-data.hacker_news.full
        GROUP BY parent 
        HAVING Count(id) > 10 """